In [28]:
import pandas as pd
import zipfile
from itertools import product
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output

hero_df = pd.read_csv("C:/Users/karth/Documents/Dashboard/Dashboard/archive (2)/06_hero_vs.csv")
hero_df
#Filtering to positions I care about illustrating, which is top
filtered_df_2024 = hero_df[hero_df['matchType'].str.startswith('2024')]
filtered_df_2024_top = filtered_df_2024[filtered_df_2024['hero1_position'].str.startswith('TOP')]
filtered_df_2024_top

# Assume top_5_hero1 and top_5_hero2 are lists of your 5 heroes for hero1 and hero2
top_5_hero1 = filtered_df_2024_top['hero1_name'].value_counts().head(10).index.tolist()
top_5_hero2 = filtered_df_2024_top['hero2_name'].value_counts().head(10).index.tolist()

# Create all combinations of hero1 and hero2
hero_combinations = list(product(top_5_hero1, top_5_hero2))

# Initialize an empty list to store results
results = []

# Loop through each combination to calculate averages
for hero1, hero2 in hero_combinations:
    # Filter the DataFrame for this combination
    subset = filtered_df_2024_top[
        (filtered_df_2024_top['hero1_name'] == hero1) & 
        (filtered_df_2024_top['hero2_name'] == hero2)
    ]
    
    # If the subset is not empty, calculate the averages
    if not subset.empty:
        avg_minutes_played = subset['minutes_played'].mean()
        avg_matches_won_1 = subset['matches_won_1'].mean()
        avg_matches_won_2 = subset['matches_won_2'].mean()
        avg_hero1_cs = subset['hero1_CS'].mean()
        avg_hero2_cs = subset['hero2_CS'].mean()
        avg_hero1_kills = subset['hero1_kills'].mean()
        avg_hero2_kills = subset['hero2_kills'].mean()
        avg_hero1_gold = subset['hero1_gold'].mean()
        avg_hero2_gold = subset['hero2_gold'].mean()

        
        # Append the results as a dictionary
        results.append({
            'hero1_name': hero1,
            'hero2_name': hero2,
            'avg_minutes_played': avg_minutes_played,
            'avg_matches_won_1': avg_matches_won_1,
            'avg_matches_won_2': avg_matches_won_2,
            'avg_hero1_cs': avg_hero1_cs,
            'avg_hero2_cs': avg_hero2_cs,
            'avg_hero1_kills': avg_hero1_kills,
            'avg_hero2_kills': avg_hero2_kills,
            'avg_hero1_gold': avg_hero1_gold,
            'avg_hero2_gold': avg_hero2_gold
        })

# Convert the results list into a DataFrame
combinations_df = pd.DataFrame(results)

# Display the resulting DataFrame
#print(combinations_df)

contingency_kills = pd.crosstab(
    index=combinations_df['hero1_name'],  # Rows (Hero1)
    columns=combinations_df['hero2_name'],  # Columns (Hero2)
    values=combinations_df['avg_hero1_kills'],  # Data to aggregate
    aggfunc='sum'  # Aggregation function
)
contingency_avg_minutes_played = pd.crosstab(
    index=combinations_df['hero1_name'],  # Rows (Hero1)
    columns=combinations_df['hero2_name'],  # Columns (Hero2)
    values=combinations_df['avg_minutes_played'],  # Data to aggregate
    aggfunc='sum'  # Aggregation function
)
contingency_avg_matches_won = pd.crosstab(
    index=combinations_df['hero1_name'],  # Rows (Hero1)
    columns=combinations_df['hero2_name'],  # Columns (Hero2)
    values=combinations_df['avg_matches_won_1'],  # Data to aggregate
    aggfunc='sum'  # Aggregation function
)
contingency_avg_cs = pd.crosstab(
    index=combinations_df['hero1_name'],  # Rows (Hero1)
    columns=combinations_df['hero2_name'],  # Columns (Hero2)
    values=combinations_df['avg_hero1_cs'],  # Data to aggregate
    aggfunc='sum'  # Aggregation function
)
contingency_avg_gold= pd.crosstab(
    index=combinations_df['hero1_name'],  # Rows (Hero1)
    columns=combinations_df['hero2_name'],  # Columns (Hero2)
    values=combinations_df['avg_hero2_gold'],  # Data to aggregate
    aggfunc='sum'  # Aggregation function
)

#Creating app to display possible contigency tables
app = dash.Dash(__name__)
# Layout of the Dashboard
app.layout = html.Div([
    html.H1(
    "Interactive Contingency Tables Dashboard (Top 10 Heroes)",
    style={
        'color': 'white',  # Title text color
        'text-align': 'center',  # Center-align the title
        'background-color': 'black',  # Optional: Add a background color for contrast
        'padding': '10px'  # Optional: Add some padding for better spacing
    }
),
    

    # Dropdown to select metric
    dcc.Dropdown(
        id='metric-dropdown',
        options=[
            {'label': 'Average Kills', 'value': 'Average Kills'},
            {'label': 'Average Gold Earned', 'value': 'Average Gold Earned'},
            {'label': 'Average Minutes Played', 'value': 'Average Minutes Played'}, 
            {'label': 'Average Matches Won', 'value': 'Average Matches Won'}, 
            {'label': 'Average CS', 'value': 'Average CS'}, 
        ],
        value='kills',  # Default value
        style={'width': '50%'}
    ),
    
    # Display the selected contingency table as a graph
    dcc.Graph(id='contingency-table')
])

# Callback to update the contingency table based on dropdown selection
@app.callback(
    Output('contingency-table', 'figure'),
    [Input('metric-dropdown', 'value')]
)
def update_table(selected_metric):
    if selected_metric == 'Average Kills':
        # Create the contingency table for kills
        fig = go.Figure(data=[go.Heatmap(
            z=contingency_kills.values,
            x=contingency_kills.columns,
            y=contingency_kills.index,
            colorscale='Viridis'
        )])
    elif selected_metric == 'Average Gold Earned':
        # Create the contingency table for gold
        fig = go.Figure(data=[go.Heatmap(
            z=contingency_avg_gold.values,
            x=contingency_avg_gold.columns,
            y=contingency_avg_gold.index,
            colorscale='Viridis'
        )])
    elif selected_metric == 'Average Minutes Played':
        # Create the contingency table for gold
        fig = go.Figure(data=[go.Heatmap(
            z=contingency_avg_minutes_played.values,
            x=contingency_avg_minutes_played.columns,
            y=contingency_avg_minutes_played.index,
            colorscale='Viridis'
        )])
    elif selected_metric == 'Average Matches Won':
        # Create the contingency table for gold
        fig = go.Figure(data=[go.Heatmap(
            z=contingency_avg_matches_won.values,
            x=contingency_avg_matches_won.columns,
            y=contingency_avg_matches_won.index,
            colorscale='Viridis'
        )])   
    else:
        # Create the contingency table for gold
        fig = go.Figure(data=[go.Heatmap(
            z=contingency_avg_cs.values,
            x=contingency_avg_cs.columns,
            y=contingency_avg_cs.index,
            colorscale='Viridis'
        )])     

    # Add labels and title
    fig.update_layout(
        title=f'Contingency Table for {selected_metric.capitalize()}',
        xaxis_title='Hero2 Name',
        yaxis_title='Hero1 Name',
        xaxis={'showgrid': False},
        yaxis={'showgrid': False}
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[28], line 182, in update_table(selected_metric=None)
    173     fig = go.Figure(data=[go.Heatmap(
    174         z=contingency_avg_cs.values,
    175         x=contingency_avg_cs.columns,
    176         y=contingency_avg_cs.index,
    177         colorscale='Viridis'
    178     )])     
    180 # Add labels and title
    181 fig.update_layout(
--> 182     title=f'Contingency Table for {selected_metric.capitalize()}',
        fig = Figure({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
                             [0.2222222222222222, '#3e4989'], [0.3333333333333333,
                             '#31688e'], [0.4444444444444444, '#26828e'],
                             [0.5555555555555556, '#1f9e89'], [0.6666666666666666,
                             '#35b779'], [0.7777777777777778, '#6ece58']